In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [4]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/SemEval2016-Task6-raw-annotations-stance.csv')


In [1]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[=================================================-] 99.9% 1660.7/1662.8MB downloaded


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
import re

In [7]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Stance'] = encoder.fit_transform(df['Stance'])

In [13]:
def rnn_wordvec(df, wv):


    # Separate the majority and minority classes
    df_majority = df[df['Stance'] == 0]
    df_minority = df[df['Stance'] == 1]

    # Resample the majority class to match the number of samples in the minority class
    df_majority_resampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)

    # Concatenate the resampled majority class with the minority class
    df_resampled = pd.concat([df_majority_resampled, df_minority])

    # Shuffle the rows of the resampled dataset
    df_resampled = df_resampled.sample(frac=1, random_state=42)
    df = df_resampled

    def clean_data(review):
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        return review

    df['Tweet'] = df['Tweet'].apply(clean_data)

    def remove_stop_words(review):
        review_minus_sw = []
        stop_words = stopwords.words('english')
        review = review.split()
        review = [review_minus_sw.append(word) for word in review if word not in stop_words]
        review = ' '.join(review_minus_sw)
        return review

    df['Tweet'] = df['Tweet'].apply(remove_stop_words)

    def lematize(review):
        lemmatizer = WordNetLemmatizer()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review]
        review = ' '.join(review)
        return review

    df['Tweet'] = df['Tweet'].apply(lematize)

    message = df['Tweet']
    stance = df['Stance']

    words = []
    for sent in message:
        sent_token = sent_tokenize(sent)
        for sent in sent_token:
            words.append(simple_preprocess(sent)) 

    word_indeces = [[wv.vocab[word].index for word in sentence if word in wv.vocab] for sentence in words]
    input_data = pad_sequences(word_indeces, maxlen=100, padding='post')
    print("IP DATA:",input_data)
    X_train, X_test , y_train, y_test = train_test_split(input_data, stance, test_size=0.20)

    max_length = 100
    vocab_size = len(wv.vocab)

    def define_rnn_model():
        model = Sequential()
        model.add(Embedding(3000000, 300, input_length=max_length, weights=[wv.vectors]))
        model.add(LSTM(128))
        model.add(Dense(10, activation='relu'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    # call the define_rnn_model function to create the model object
    model = define_rnn_model()

    # print the model summary
    model.summary()

    # calling adam optimizer
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    # visualizing each eppoch
    history = model.fit(X_train, y_train, batch_size = 32, epochs=10, validation_data=(X_test, y_test))

    # caluclating loss and accuracy
    loss, accuracy = model.evaluate(X_test,y_test)

    # printing accuracy
    print('Testing Accuracy is {} '.format(accuracy*100))

    y_prob = model.predict(X_test)
    y_predict = (y_prob>=0.5).astype(int)
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_predict,target_names = ["Against","Favor"]))
rnn_wordvec(df,wv)


IP DATA: [[ 17985   2213    274 ...      0      0      0]
 [381604  14033   2747 ...      0      0      0]
 [   292   4282    292 ...      0      0      0]
 ...
 [  3573  12751     45 ...      0      0      0]
 [766185    155   1170 ...      0      0      0]
 [210121 435105    202 ...      0      0      0]]


ResourceExhaustedError: ignored